In [2]:
import re
import string
import graphlab as gl # this import will be assumed from now on
import pandas as pd
import funcy as fp

gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 24)

In [ ]:
stories_sf = gl.load_sframe("hacker_news_stories.sframe") #gl.load_sframe("http://s3.amazonaws.com/dato-datasets/hacker_news/stories_with_text.sframe")

In [58]:
re_print = re.compile(r'[^' + string.printable + ']')
re_contraction = re.compile(r'(\w+)\s+(t|ve|d|ll|m|re)\b')

def combine_and_clean(row):
    cat = row["title"] + ' ' + row["text"]
    ascii = re_print.sub(' ', cat)
    # removing the odd charaters resulted in invalid space-filled contractions
    fixed = re_contraction.sub(lambda m: "'".join(m.groups()), ascii)
    return fixed

stories_sf["text_title"] = stories_sf.apply(combine_and_clean)

I am using [spaCy](https://honnibal.github.io/spaCy/) to do tokenization and POS tagging. It did not work well with GraphLab and so I have convert the dataframe to a pandas one and process it locally. spaCy is a fast but only uses a single core. Since processing the entire dataset is embarrassingly parallel I'll be using [joblib](https://pythonhosted.org/joblib/) to use all the cores to process all the documents.

(This takes ~19GB of memory to do the entire dataset, so use a sample of the dataset if you are following along and have less memory.)

In [12]:
#Use all of it!
sample = stories_sf #  stories_sf.sample(0.4, seed=42)

In [13]:
del sample["text"]
del sample["title"]
df = sample.to_dataframe()

In [4]:
from joblib import Parallel, delayed, cpu_count

def _series_chunks(s, n_jobs):
    if n_jobs < 0:
        # so, have n chunks if we are using all n cores/cpus = cpu_count() + 1 + n_jobs
        n_jobs = cpu_count() + 1 + n_jobs
    n = len(s)
    n_chunks = int(n / n_jobs)
    return (s.iloc[ilocs] for ilocs in fp.chunks(n_chunks, range(n)))

def series_pmap(s, f, n_jobs=-1):
    if n_jobs == 0:
        return s.map(f)
    return pd.concat(Parallel(n_jobs=n_jobs)(delayed(series_pmap)(sub_series, f, n_jobs=0) \
                                                 for sub_series in _series_chunks(s, n_jobs)))

In [5]:
from spacy.en import English
nlp = English()

In [6]:
def tokenize(s):
    tokens = nlp(unicode(s))
    # the native spaCy tuples don't pickle so we extract the data into our own structure
    # Also, since graphlab doesn't handle namedtuples (or tuples) so we have to use a list :(
    return [[t.orth_.lower(), t.lemma_.lower(), t.pos, t.tag] for t in tokens if t.orth_.strip()]

In [18]:
%%time
tokens = series_pmap(df['text_title'], tokenize)

CPU times: user 3min 21s, sys: 39.6 s, total: 4min 1s
Wall time: 25min 10s


In [19]:
stories_sf["tokens"] = gl.SArray(tokens)

Now that we have tokenized and tagged the docs we now want to do some simple phrase detection (aka [collocation extraction](https://en.wikipedia.org/wiki/Collocation_extraction)) using ngrams. We'll be using gensim's [Phrase](http://radimrehurek.com/gensim/models/phrases.html) class to do that. It takes a stream of sentences. We only want to consider certain types of words as part of phrases so we will use the others (including punctiontion) as a sentence chunk boundary. Meaning, a single sentence will be segmented into potentialy various chunks.  I opted to write my own chunker since it was easy and spaCy doesn't have one ([nltk](http://www.nltk.org/) does however).

In [39]:
import spacy.parts_of_speech as pos

DEFAULT_POS = set([pos.NOUN, pos.VERB, pos.ADV, pos.ADJ])
DEFAULT_STOPWORDS = gl.text_analytics.stopwords() | set(['pm','am', "'re", "'ve", "n't"])

In [19]:
def pos_name(pos_id):
    if pos_id == pos.NOUN: return 'NOUN'
    if pos_id == pos.VERB: return 'VERB'
    if pos_id == pos.ADV: return 'ADV'
    if pos_id == pos.ADJ: return 'ADJ'
    return 'OTHER'

In [33]:
 def chunk(tokens, allowed_pos=DEFAULT_POS):
    split = []
    for token in tokens:
        orth, lemma, pos, tag = token
        if pos in allowed_pos:
            split.append(lemma +  '/' + pos_name(pos))
        else:
            # break detected!
            if len(split) > 1:
                yield split
            if len(split) > 0:
                split = []

In [18]:
tokens = pd.Series(stories_sf['tokens'])

In [30]:
def chunk_doc(tokens):
    return list(chunk(tokens))

In [38]:
%%time
chunked_docs = tokens.map(chunk_doc)

0%                          100%
[##############################] | ETA[sec]: 0.000 

CPU times: user 52.4 s, sys: 4.7 s, total: 57.1 s
Wall time: 2min 1s



Total time elapsed: 117.968 sec


In [41]:
contiguous_chunks = fp.cat(chunked_docs)

In [43]:
from gensim.models.phrases import Phrases

In [44]:
%%time
# train chunker now
bigram = Phrases(contiguous_chunks)
trigram = Phrases(bigram[contiguous_chunks])


[INFO] collecting all words and their counts
[INFO] PROGRESS: at sentence #0, processed 0 words and 0 word types
[INFO] PROGRESS: at sentence #10000, processed 17397 words and 19185 word types
[INFO] PROGRESS: at sentence #20000, processed 34388 words and 33793 word types
[INFO] PROGRESS: at sentence #30000, processed 52011 words and 46907 word types
[INFO] PROGRESS: at sentence #40000, processed 69618 words and 59534 word types
[INFO] PROGRESS: at sentence #50000, processed 87074 words and 70893 word types
[INFO] PROGRESS: at sentence #60000, processed 104624 words and 81860 word types
[INFO] PROGRESS: at sentence #70000, processed 122055 words and 92821 word types
[INFO] PROGRESS: at sentence #80000, processed 139804 words and 104420 word types
[INFO] PROGRESS: at sentence #90000, processed 157386 words and 114209 word types
[INFO] PROGRESS: at sentence #100000, processed 174403 words and 123936 word types
[INFO] PROGRESS: at sentence #110000, processed 191929 words and 134046 word t

CPU times: user 7min 59s, sys: 18.6 s, total: 8min 17s
Wall time: 8min 2s


In [45]:
%%time
tokenized_and_filtered_docs = chunked_docs.map(fp.cat)

CPU times: user 5.62 s, sys: 731 ms, total: 6.35 s
Wall time: 16.6 s


In [46]:
def extract_chunks(doc, min_len=3):
    return [w for w in trigram[bigram[doc]] if len(w) >= min_len and w not in DEFAULT_STOPWORDS]

In [59]:
%%time
processed_docs = series_pmap(tokenized_and_filtered_docs, extract_chunks)
stories_sf["processed"] = gl.SArray(processed_docs)
stories_sf.save("hn_annotated_processed.sframe")

In [57]:
%%time
dictionary = Dictionary(processed_docs)
dictionary.save('annotated_unfiltered.dict')

[INFO] adding document #0 to Dictionary(0 unique tokens: [])
[INFO] adding document #10000 to Dictionary(74538 unique tokens: [u'indie/ADJ_game/NOUN_studio/NOUN', u'hearted/ADJ', u'patient/ADJ', u'follow/connect/VERB', u'privacy/NOUN_setting/NOUN']...)
[INFO] adding document #20000 to Dictionary(96569 unique tokens: [u'unmanned/ADJ', u'follow/connect/VERB', u'privacy/NOUN_setting/NOUN', u'viewtext.org/NOUN', u'wilmington/NOUN']...)
[INFO] adding document #30000 to Dictionary(127157 unique tokens: [u'unmanned/ADJ', u'follow/connect/VERB', u'privacy/NOUN_setting/NOUN', u'viewtext.org/NOUN', u'wilmington/NOUN']...)
[INFO] adding document #40000 to Dictionary(220374 unique tokens: [u'underlay/NOUN', u'patient/ADJ', u'follow/connect/VERB', u'wilmington/NOUN', u'organic/ADJ_search/NOUN']...)
[INFO] adding document #50000 to Dictionary(262038 unique tokens: [u'omgpop/NOUN', u'underlay/NOUN', u'patient/ADJ', u'@persist/NOUN', u'follow/connect/VERB']...)
[INFO] adding document #60000 to Diction

CPU times: user 45.1 s, sys: 6.32 s, total: 51.4 s
Wall time: 52 s
